0 导入包

In [1]:
from datasets import load_dataset

1 加载数据集

In [2]:
# 加载数据集
dataset = load_dataset("json", data_files="1qa_data/qa_data.jsonl", split="train")
dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 4260
})

2 qa -> prompt

In [3]:
# 构造prompt数据
# 按照每一个模型自己的规则处理成prompt形式
# 训练时候规定了用户开头是什么词，模型开头是什么词。这些词叫做特殊词，会被大模型过滤掉。
# 这些prompt写法在其他官方教程和special_tokens_map这个文件中

# 1 构造一个函数
def create_prompt(x1,x2):
    # 定义整体格式
    d_prompt = '''<bos><start_of_turn>user
根据输入金额和产品关键字，推荐购物套餐。 {x_1}<end_of_turn>'
<start_of_turn>model
{x_2} <end_of_turn><eos>'''

    # 提示词批量替换
    prompt = d_prompt.format(
        x_1=x1,
        x_2=x2,
    )
    return prompt

# 2 循环处理每一行数据
prompt_list =[]
for i in range(len(dataset)):
    prompt_list.append(
        create_prompt(x1=dataset[i]['input'], x2=dataset[i]['output'])
    )
    # break # 循环1次试试

# 3 添加新的一列数据
dataset = dataset.add_column("prompt", prompt_list)
dataset

Dataset({
    features: ['input', 'output', 'prompt'],
    num_rows: 4260
})

In [4]:
dataset[0]

{'input': '价格区间：11元，商品名称：农夫山泉东方树叶',
 'output': '总价：11.7元，3个农夫山泉东方树叶',
 'prompt': "<bos><start_of_turn>user\n根据输入金额和产品关键字，推荐购物套餐。 价格区间：11元，商品名称：农夫山泉东方树叶<end_of_turn>'\n<start_of_turn>model\n总价：11.7元，3个农夫山泉东方树叶 <end_of_turn><eos>"}

3 保存数据

In [5]:
# 保存数据
dataset.save_to_disk("2prompt_data")

Saving the dataset (0/1 shards):   0%|          | 0/4260 [00:00<?, ? examples/s]